In [1]:
library(ggplot2) # Data visualization
library(readr) # CSV file I/O, e.g. the read_csv function
library(tidyverse)

#Loading datasets
train_dataset = read.csv('../input/train.csv')
train_dataset_PassangerID = NULL
train_dataset$Origin = 1
test_dataset = read.csv('../input/test.csv')
test_dataset$Origin = 2
test_dataset$Survived = 3
full_dataset = rbind(train_dataset, test_dataset)
str(train_dataset)
str(test_dataset)


── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ purrr   0.2.4     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


'data.frame':	891 obs. of  13 variables:
 $ PassengerId: int  1 2 3 4 5 6 7 8 9 10 ...
 $ Survived   : int  0 1 1 1 0 0 0 0 1 1 ...
 $ Pclass     : int  3 1 3 1 3 3 1 3 3 2 ...
 $ Name       : Factor w/ 891 levels "Abbing, Mr. Anthony",..: 109 191 358 277 16 559 520 629 417 581 ...
 $ Sex        : Factor w/ 2 levels "female","male": 2 1 1 1 2 2 2 2 1 1 ...
 $ Age        : num  22 38 26 35 35 NA 54 2 27 14 ...
 $ SibSp      : int  1 1 0 1 0 0 0 3 0 1 ...
 $ Parch      : int  0 0 0 0 0 0 0 1 2 0 ...
 $ Ticket     : Factor w/ 681 levels "110152","110413",..: 524 597 670 50 473 276 86 396 345 133 ...
 $ Fare       : num  7.25 71.28 7.92 53.1 8.05 ...
 $ Cabin      : Factor w/ 148 levels "","A10","A14",..: 1 83 1 57 1 1 131 1 1 1 ...
 $ Embarked   : Factor w/ 4 levels "","C","Q","S": 4 2 4 4 4 3 4 4 4 2 ...
 $ Origin     : num  1 1 1 1 1 1 1 1 1 1 ...
'data.frame':	418 obs. of  13 variables:
 $ PassengerId: int  892 893 894 895 896 897 898 899 900 901 ...
 $ Pclass     : int  3 3 2 3 3 3 3 

In [2]:
# Split name into title and laste name
full_dataset$LastName = substring(full_dataset$Name, 1, regexpr(',', full_dataset$Name)-1)
full_dataset$Temp    = substring(full_dataset$Name, regexpr(',', full_dataset$Name)+2)
full_dataset$Title  = substring(full_dataset$Temp, 1, regexpr(' ', full_dataset$Temp))
full_dataset$Title = ifelse(full_dataset$Title == "Capt. " , "Mr. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Col. " , "Mr. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Dr. " , "Mr. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Jonkheer. " , "Mr. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Don. " , "Mr. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Major. " , "Mr. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Master. " , "Mr. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Rev. " , "Mr. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Sir. " , "Mr. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "the " , "Mr. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Lady. " , "Mrs. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Mme. " , "Mrs. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Dona. " , "Mrs. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Mlle. " , "Ms. " , full_dataset$Title)
full_dataset$Title = ifelse(full_dataset$Title == "Miss. " , "Ms. " , full_dataset$Title)
table(full_dataset$Title)
full_dataset$Title = factor(full_dataset$Title, levels = c("Mr. ", "Mrs. ", "Ms. "), labels = c(1,2,3))
full_dataset$Title = strtoi(full_dataset$Title)

# Remove Name 
full_dataset$Name = NULL

# Remove ticket
full_dataset$Ticket = NULL


 Mr.  Mrs.   Ms.  
  845   200   264 

In [3]:
# Change Sex to 0 or 1
full_dataset$Sex = factor(full_dataset$Sex, levels = c('male', 'female'), labels = c(1,2))
full_dataset$Sex = strtoi(full_dataset$Sex)

# Change Embarked to 1,2 or 3
full_dataset$Embarked = factor(full_dataset$Embarked, levels = c('S', 'C', 'Q'), labels = c(1,2,3))
full_dataset$Embarked = strtoi(full_dataset$Embarked)
table(full_dataset$Embarked)
full_dataset$Embarked = ifelse(is.na(full_dataset$Embarked), 2, full_dataset$Embarked)


# Cabin Section - 0 indicate unknown cabin
full_dataset$CabinSection = substr(full_dataset$Cabin, 1, 1)
full_dataset$CabinSection = ifelse(full_dataset$CabinSection == '', 'Z', full_dataset$CabinSection)
full_dataset$CabinSection = factor(full_dataset$CabinSection, 
                          levels = c('A','B','C','D','E','F','G','T','Z'), 
                          labels = c( 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 0 ))
full_dataset$CabinSection = strtoi(full_dataset$CabinSection)

# Remove original Cabin
full_dataset$Cabin = NULL

# Add family size variable
full_dataset$FamilySize = full_dataset$SibSp + full_dataset$Parch + 1
full_dataset$FsizeD[full_dataset$FamilySize == 1] = '1'
full_dataset$FsizeD[full_dataset$FamilySize < 5 & full_dataset$FamilySize > 1 ] = '2'
full_dataset$FsizeD[full_dataset$FamilySize > 4] = '3'
full_dataset$FsizeD = strtoi(full_dataset$FsizeD)

#full_dataset$FsizeD = factor(full_dataset$FsizeD, levels = c('A', 'B', 'C'), labels = c(1,2,3))



  1   2   3 
914 270 123 

In [4]:
# Estimating Age missing values
#table(full_dataset$Age)
#sum(is.na(full_dataset$Age))


# Age has missing values - filling in with the mean
# other options is to use MICE!!!
full_dataset$Age = ifelse(is.na(full_dataset$Age),
                     ave(full_dataset$Age, FUN = function(x) mean(x, na.rm = TRUE)),
                     full_dataset$Age)               
full_dataset$Fare = ifelse(is.na(full_dataset$Fare),
                     ave(full_dataset$Fare, FUN = function(x) mean(x, na.rm = TRUE)),
                     full_dataset$Fare) 
                           
# Refactor all variables  
train_dataset = full_dataset[1:891,]
train_dataset = subset(train_dataset, select=c('Age','FsizeD', 'Sex' , 'Pclass', 'Survived'))
train_dataset[, 1:4] = scale(train_dataset[, 1:4])
#train_dataset = subset(train_dataset, select=c('Pclass','Age','Fare','SibSp', 'Parch', 
#                                               'FsizeD', 'Sex' , 'Title', 'Embarked',
#                                               'CabinSection',
#                                               'Survived'))
#train_dataset[, 1:10] = scale(train_dataset[, 1:10])
head(train_dataset, 5)
                           
test_dataset = full_dataset[892:1309,]
str(test_dataset)
test_dataset = subset(test_dataset, select=c('PassengerId','Age','FsizeD', 'Sex', 'Pclass')) 
test_dataset[, 2:5] = scale(test_dataset[, 2:5])
#test_dataset = subset(test_dataset, select=c('PassengerId', 'Pclass','Age','Fare', 
#                                             'SibSp', 'Parch', 
#                                             'FsizeD', 'Sex' , 'Title', 'Embarked', 'CabinSection'))                          
#test_dataset[, 2:11] = scale(test_dataset[, 2:11])
head(test_dataset, 5)
                     
#split to test and train
#library(caTools)
#set.seed(123)
#split = sample.split(dataset$Survived, SplitRatio = 0.8)
#training_set = subset(dataset, split == TRUE)
#test_set = subset(dataset, split == FALSE)


Age,FsizeD,Sex,Pclass,Survived
-0.5949198,0.8552935,-0.737281,0.8269128,0
0.6356395,0.8552935,1.354813,-1.5652278,1
-0.2872799,-0.7490571,1.354813,0.8269128,1
0.4049096,0.8552935,1.354813,-1.5652278,1
0.4049096,-0.7490571,-0.737281,0.8269128,0


'data.frame':	418 obs. of  16 variables:
 $ PassengerId : int  892 893 894 895 896 897 898 899 900 901 ...
 $ Survived    : num  3 3 3 3 3 3 3 3 3 3 ...
 $ Pclass      : int  3 3 2 3 3 3 3 2 3 3 ...
 $ Sex         : int  1 2 1 1 2 1 2 1 2 1 ...
 $ Age         : num  34.5 47 62 27 22 14 30 26 18 21 ...
 $ SibSp       : int  0 1 0 0 1 0 0 1 0 2 ...
 $ Parch       : int  0 0 0 0 1 0 0 1 0 0 ...
 $ Fare        : num  7.83 7 9.69 8.66 12.29 ...
 $ Embarked    : num  3 1 3 1 1 1 3 1 2 1 ...
 $ Origin      : num  2 2 2 2 2 2 2 2 2 2 ...
 $ LastName    : chr  "Kelly" "Wilkes" "Myles" "Wirz" ...
 $ Temp        : chr  "Mr. James" "Mrs. James (Ellen Needs)" "Mr. Thomas Francis" "Mr. Albert" ...
 $ Title       : int  1 2 1 1 2 1 3 1 2 1 ...
 $ CabinSection: int  0 0 0 0 0 0 0 0 0 0 ...
 $ FamilySize  : num  1 2 1 1 3 1 1 3 1 3 ...
 $ FsizeD      : int  1 2 1 1 2 1 1 2 1 2 ...


,PassengerId,Age,FsizeD,Sex,Pclass
892,892,0.3409393,-0.7554579,-0.7550242,0.8724364
893,893,1.3302134,0.9514686,1.3212923,0.8724364
894,894,2.5173423,-0.7554579,-0.7550242,-0.3154412
895,895,-0.2526252,-0.7554579,-0.7550242,0.8724364
896,896,-0.6483348,0.9514686,1.3212923,0.8724364


In [5]:
# Log reg
# ---------                           
classifier = glm(Survived ~ ., 
                family = binomial,
                data = train_dataset)                     
test_prod_pred = predict(classifier, type='response', newdata = test_dataset)
Y_pred = ifelse(test_prod_pred > 0.5, 1, 0)
test_dataset$Survived = Y_pred
summary(classifier)
result = subset(test_dataset, select=c('PassengerId' , 'Survived'))                          
write.csv(result, file = "MyResults23-LogReg.csv",row.names=FALSE)

str(test_dataset)


Call:
glm(formula = Survived ~ ., family = binomial, data = train_dataset)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-2.673  -0.609  -0.428   0.617   2.405  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -0.64580    0.09018  -7.161 7.99e-13 ***
Age         -0.49185    0.09937  -4.950 7.43e-07 ***
FsizeD      -0.25477    0.09516  -2.677  0.00742 ** 
Sex          1.32357    0.09543  13.870  < 2e-16 ***
Pclass      -1.00085    0.10026  -9.982  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1186.66  on 890  degrees of freedom
Residual deviance:  797.84  on 886  degrees of freedom
AIC: 807.84

Number of Fisher Scoring iterations: 5


'data.frame':	418 obs. of  6 variables:
 $ PassengerId: int  892 893 894 895 896 897 898 899 900 901 ...
 $ Age        : num  0.341 1.33 2.517 -0.253 -0.648 ...
 $ FsizeD     : num  -0.755 0.951 -0.755 -0.755 0.951 ...
 $ Sex        : num  -0.755 1.321 -0.755 -0.755 1.321 ...
 $ Pclass     : num  0.872 0.872 -0.315 0.872 0.872 ...
 $ Survived   : num  0 0 0 0 1 0 1 0 1 0 ...


In [6]:
# KNN classification
# ------------------                           
library(class)
y_pred = knn(train = train_dataset[, 1:4],
             test = test_dataset[, 2:5],
             cl = train_dataset[, 5],
             k = 25)                       
         
test_dataset$Survived = y_pred  
result = subset(test_dataset, select=c('PassengerId' , 'Survived'))
result[150:160, ]  
write.csv(result, file = "MyResults24-KNN.csv",row.names=FALSE)

                      


,PassengerId,Survived
1041,1041,0
1042,1042,1
1043,1043,0
1044,1044,0
1045,1045,1
1046,1046,0
1047,1047,0
1048,1048,1
1049,1049,1
1050,1050,0


In [7]:
#SVM classification
#------------------
library("e1071")
svm_model = svm(Survived ~ ., 
                data=train_dataset,
                type='C-classification',
               kernel='radial')
summary(svm_model)


test_dataset$Survived = predict(svm_model,test_dataset)
result = subset(test_dataset, select=c('PassengerId' , 'Survived'))                          
write.csv(result, file = "MyResults25-SVM.csv",row.names=FALSE)



Call:
svm(formula = Survived ~ ., data = train_dataset, type = "C-classification", 
    kernel = "radial")


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  1 
      gamma:  0.25 

Number of Support Vectors:  370

 ( 188 182 )


Number of Classes:  2 

Levels: 
 0 1


